Inleiding:>>

In [2]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

In [3]:
#daan
df_land = pd.read_excel('dataset1.xls', sheet_name=1, skiprows=2,nrows=43)
colormap = {
    'Wheat & Rye (Bread)':'yellow',
    'Maize (Meal)':'yellow',
    'Barley (Beer)':'yellow',
    'Oatmeal':'yellow',
    'Rice':'yellow',
    'Potatoes':'yellow',
    'Cassava':'yellow',
    'Cane Sugar':'pink',
    'Beet Sugar':'pink',
    'Other Pulses':'pink',
    'Peas':'brown',
    'Nuts':'brown',
    'Groundnuts':'brown',
    'Soymilk':'lightgreen',
    'Tofu':'lightgreen',
    'Soybean Oil':'black',
    'Palm Oil':'black',
    'Sunflower Oil':'black',
    'Rapeseed Oil':'black',
    'Olive Oil':'black',
    'Tomatoes':'green',
    'Onions & Leeks':'green',
    'Root Vegetables':'green',
    'Brassicas':'green',
    'Other Vegetables':'green',
    'Citrus Fruit':'orange',
    'Bananas':'orange',
    'Apples':'orange',
    'Berries & Grapes':'orange',
    'Wine':'orange',
    'Other Fruit':'orange',
    'Coffee':'gold',
    'Dark Chocolate':'gold',
    'Bovine Meat (beef herd)':'red',
    'Bovine Meat (dairy herd)':'red',
    'Lamb & Mutton':'red',
    'Pig Meat':'red',
    'Poultry Meat':'red',
    'Milk':'grey',
    'Cheese':'grey',
    'Eggs':'grey',
    'Fish (farmed)':'blue',
    'Crustaceans (farmed)':'blue'
}
color_sequence = ['red', 'green', 'blue', 'orange', 'purple']
# fig = px.histogram(df_land, x='Product', y='Mean.5', color_discrete_map=colormap)
fig = px.histogram(df_land, x='Product', y='Mean.5', color='Product', color_discrete_map=colormap)
fig.update_layout(
    font=dict(
        family='Lato, "Helvetica Neue", Helvetica, Arial, "Liberation Sans", sans-serif',
        size=13,
    ),
    yaxis=dict(title='Average Freshwater Withdrawals (L/NU)'),
    xaxis=dict(title='Products'),
    title='Freshwater Withdrawals (L/NU)'
)
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset1.xls'

fig1: Amount of freshwater withdrawals per nutritional units

In figure 1 you can see the different amount of liters water used for a nutritional unit. In the figure you can also see that nuts and cheese use alot of water for the amount of nutritions that they give. Most of the meat substitutes are made from wheat, soy and different kinds of fungi(https://www.milieucentraal.nl/eten-en-drinken/milieubewust-eten/vleesvervangers/). The figure shows that there is a substatial difference between amount of water used for the different kinds of meat and the water usage from wheat and soymilk. The only meat that doesn't use more than double the amount of water as the wheat is poultry meat. 

In [4]:
df_land = pd.read_excel('dataset1.xls', sheet_name=0, skiprows=2,nrows=43)

fig = px.histogram(df_land, x='Product', y='Mean.5', color='Product', color_discrete_map=colormap)
fig.update_layout(
    font=dict(
        family='Lato, "Helvetica Neue", Helvetica, Arial, "Liberation Sans", sans-serif',
        size=13,
    ),
    xaxis=dict(title='Average Freshwater Withdrawals (L/FU)'),
    yaxis=dict(title='Products'),
    title = 'Freshwater Withdrawals (L/FU)'
)
fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'dataset1.xls'

fig2: Amount of freshwater withdrawals per Fixture unit

In [6]:
#Mauro
non_vegan_products = ['Barley (Beer)','Cane Sugar','Milk','Cheese','Eggs','Fish (farmed)','Crustaceans (farmed)','Bovine Meat (beef herd)','Bovine Meat (dairy herd)','Lamb & Mutton','Pig Meat','Poultry Meat']
meat_products = ['Bovine Meat (beef herd)', 'Bovine Meat (dairy herd)', 'Lamb & Mutton', 'Pig Meat','Poultry Meat','Fish (farmed)','Crustaceans (farmed)']
dairy_products = ['Milk','Cheese','Eggs']
other_non_vegan_products = ['Cane Sugar','Crustaceans (farmed)','Barley (Beer)']
df_all_products = pd.read_excel("dataset1.xls", sheet_name="Results - Retail Weight", skiprows=2,nrows=43, index_col=None, na_values=["NA"])

#non vegan products
non_vegan = df_all_products["Product"].isin(non_vegan_products)
meat = df_all_products["Product"].isin(meat_products)
dairy = df_all_products["Product"].isin(dairy_products)
other_non_vegan = df_all_products["Product"].isin(other_non_vegan_products)

df_non_vegan_products = df_all_products[non_vegan]
df_meat_products = df_all_products[meat]
df_dairy_products = df_all_products[dairy]
df_other_non_vegan_products = df_all_products[other_non_vegan]

#non vegan
df_non_vegan_products_land_use = df_non_vegan_products.iloc[:, [0, 3]]
df_non_vegan_products_ghg_2013 = df_non_vegan_products.iloc[:, [0, 9]]
df_non_vegan_products_ghg_2007 = df_non_vegan_products.iloc[:, [0, 15]]
df_non_vegan_products_acid = df_non_vegan_products.iloc[:, [0, 21]]
df_non_vegan_products_eutro = df_non_vegan_products.iloc[:, [0, 27]]
df_non_vegan_products_fresh_water_withdraw = df_non_vegan_products.iloc[:, [0, 33]]
df_non_vegan_products_stress_water_use = df_non_vegan_products.iloc[:, [0, 39]]

#meat products
df_meat_products_land_use = df_meat_products.iloc[:, [0, 3]]
df_meat_products_ghg_2013 = df_meat_products.iloc[:, [0, 9]]
df_meat_products_ghg_2007 = df_meat_products.iloc[:, [0, 15]]
df_meat_products_acid = df_meat_products.iloc[:, [0, 21]]
df_meat_products_eutro = df_meat_products.iloc[:, [0, 27]]
df_meat_products_fresh_water_withdraw = df_meat_products.iloc[:, [0, 33]]
df_meat_products_stress_water_use = df_meat_products.iloc[:, [0, 39]]
#dairy
df_dairy_products_land_use = df_dairy_products.iloc[:, [0, 3]]
df_dairy_products_ghg_2013 = df_dairy_products.iloc[:, [0, 9]]
df_dairy_products_ghg_2007 = df_dairy_products.iloc[:, [0, 15]]
df_dairy_products_acid = df_dairy_products.iloc[:, [0, 21]]
df_dairy_products_eutro = df_dairy_products.iloc[:, [0, 27]]
df_dairy_products_fresh_water_withdraw = df_dairy_products.iloc[:, [0, 33]]
df_dairy_products_stress_water_use = df_dairy_products.iloc[:, [0, 39]]

#other non vegan
df_other_non_vegan_products_land_use = df_other_non_vegan_products.iloc[:, [0, 3]]
df_other_non_vegan_products_ghg_2013 = df_other_non_vegan_products.iloc[:, [0, 9]]
df_other_non_vegan_products_ghg_2007 = df_other_non_vegan_products.iloc[:, [0, 15]]
df_other_non_vegan_products_acid = df_other_non_vegan_products.iloc[:, [0, 21]]
df_other_non_vegan_products_eutro = df_other_non_vegan_products.iloc[:, [0, 27]]
df_other_non_vegan_products_fresh_water_withdraw = df_other_non_vegan_products.iloc[:, [0, 33]]
df_other_non_vegan_products_stress_water_use = df_other_non_vegan_products.iloc[:, [0, 39]]


#vegan products
vegan = ~df_all_products['Product'].isin(non_vegan_products)
df_vegan_products = df_all_products[vegan]

df_vegan_products_land_use = df_vegan_products.iloc[:, [0, 3]]
df_vegan_products_ghg_2013 = df_vegan_products.iloc[:, [0, 9]]
df_vegan_products_ghg_2007 = df_vegan_products.iloc[:, [0, 15]]
df_vegan_products_acid = df_vegan_products.iloc[:, [0, 21]]
df_vegan_products_eutro = df_vegan_products.iloc[:, [0, 27]]
df_vegan_products_fresh_water_withdraw = df_vegan_products.iloc[:, [0, 33]]
df_vegan_products_stress_water_use = df_vegan_products.iloc[:, [0, 39]]



In [1]:
#sunbursts

def create_sunburst_figure(title, values):
    colors = ['darkgreen', 'darkred'] * 2  
    fig = go.Figure(go.Sunburst(
        labels=labels,
        parents=parents,
        values=values,
        marker=dict(colors=colors),
    ))
    
    fig.update_layout(title=title)
    fig.show()
categories = ['Land use', 'GHG emission 2013', 'GHG emission 2007', 'Acidification', 'Eutrophication', 'Fresh Water Withdrawal', 'Stress Water Use']

labels = ["", "Non-vegan", "Vegan","Meat", "Dairy products","Other non-vegan Products"] + list(df_meat_products["Product"]) + list(df_dairy_products["Product"]) + list(df_other_non_vegan_products["Product"]) + list(df_vegan_products["Product"])
parents = ["", "", "","Non-vegan", "Non-vegan","Non-vegan","Vegan"] + ["Meat"] * len(df_meat_products) + ["Dairy products"] * len(df_dairy_products) + ["Other non-vegan Products"] * df_other_non_vegan_products + ["Vegan"] * len(df_vegan_products)

# Land use
values = [0] * 3 + list(df_non_vegan_products_land_use["Mean"]) + list(df_vegan_products_land_use["Mean"])
create_sunburst_figure("Land use", values)

# GHG emission 2013
values = [0] * 3 + list(df_non_vegan_products_ghg_2013["Mean.1"]) + list(df_vegan_products_ghg_2013["Mean.1"])
create_sunburst_figure("GHG emission 2013", values)

# GHG emission 2007
values = [0] * 3 + list(df_non_vegan_products_ghg_2007["Mean.2"]) + list(df_vegan_products_ghg_2007["Mean.2"])
create_sunburst_figure("GHG emission 2007", values)

# Acidification
values = [0] * 3 + list(df_non_vegan_products_acid["Mean.3"]) + list(df_vegan_products_acid["Mean.3"])
create_sunburst_figure("Acidification", values)

# Eutrophication
values = [0] * 3 + list(df_non_vegan_products_eutro["Mean.4"]) + list(df_vegan_products_eutro["Mean.4"])
create_sunburst_figure("Eutrophication", values)

# Fresh Water Withdrawal
values = [0] * 3 + list(df_non_vegan_products_fresh_water_withdraw["Mean.5"]) + list(df_vegan_products_fresh_water_withdraw["Mean.5"])
create_sunburst_figure("Fresh Water Withdrawal", values)

# Stress Water Use
values = [0] * 3 + list(df_non_vegan_products_stress_water_use["Mean.6"]) + list(df_vegan_products_stress_water_use["Mean.6"])
create_sunburst_figure("Stress Water Use", values)

#Bar plots
categories = ['Land use', 'GHG emission 2013', 'GHG emission 2007', 'Acidification', 'Eutrophication', 'Fresh Water Withdrawal', 'Stress Water Use']
vegan_values = [df_vegan_products_land_use["Mean"].sum(),
                df_vegan_products_ghg_2013["Mean.1"].sum(),
                df_vegan_products_ghg_2007["Mean.2"].sum(),
                df_vegan_products_acid["Mean.3"].sum(),
                df_vegan_products_eutro["Mean.4"].sum(),
                df_vegan_products_fresh_water_withdraw["Mean.5"].sum(),
                df_vegan_products_stress_water_use["Mean.6"].sum()]
non_vegan_values = [df_non_vegan_products_land_use["Mean"].sum(),
                    df_non_vegan_products_ghg_2013["Mean.1"].sum(),
                    df_non_vegan_products_ghg_2007["Mean.2"].sum(),
                    df_non_vegan_products_acid["Mean.3"].sum(),
                    df_non_vegan_products_eutro["Mean.4"].sum(),
                    df_non_vegan_products_fresh_water_withdraw["Mean.5"].sum(),
                    df_non_vegan_products_stress_water_use["Mean.6"].sum()]
values = ['Land Use (m2/FU)', 'GHG Emissions (kg CO2eq/FU)', 'GHG Emissions (kg CO2eq/FU)', 'Acidifying Emissions (g SO2eq/FU)', 'Eutrophying Emissions (g PO43-eq/FU)', 'Freshwater Withdrawals (L/FU)', 'Stress-Weighted Water Use (L/FU)']


for category, vegan_value, non_vegan_value in zip(categories, vegan_values, non_vegan_values):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=['Vegan', 'Non-vegan'],
        y=[vegan_value, non_vegan_value],
        name=category,
        marker_color=['darkgreen','darkred']  
    ))

    fig.update_layout(
        title=f'Environmental Impact: {category}',
        xaxis_title='Product Type',
        yaxis_title= values[categories.index(category)]
    )
    fig.show()



NameError: name 'df_meat_products' is not defined